<a href="https://colab.research.google.com/github/hyeminboo/25-1-SelfDriving/blob/main/02_week3_Q_Learning/02_week3_Q-Learning_%EB%B6%80%ED%98%9C%EB%AF%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import gymnasium as gym
import numpy as np
import random
from IPython.display import clear_output
import time
import re

In [67]:
# 환경 생성
env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()
raw = env.render()

In [69]:
def render(raw):
    if isinstance(raw, list):
        raw = ''.join(raw)

    # ANSI escape 코드 제거
    ansi_escape = re.compile(r'\x1b[^m]*m')
    clean_output = ansi_escape.sub('', raw)

    return clean_output

In [70]:
print(render(raw))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [71]:
actions = env.action_space.n
actions
# 0 : down, 1 : up, 2 : right, 3 : left, 4 : pickup passenger, 5 : drop off passenger

np.int64(6)

In [72]:
observations = env.observation_space.n
observations # 25개의 택시 위치 * 5개의 승객 위치 * 4개의 목적지 위치
# ((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination

np.int64(500)

In [133]:
# hyperparameters

alpha = 0.1 # lr
gamma = 0.9 # discount factor
epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.99

In [134]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [135]:
EPISODES = 10000

In [136]:
# train

for episode in range(EPISODES):
    state, info = env.reset()
    state = int(state)

    done = False
    steps = 0
    while not done:
        steps += 1

        if random.uniform(0, 1) < epsilon: # explore
            action = env.action_space.sample()
        else: # exploit
            action = np.argmax(q_table[state])

        next_state, reward, terminated, truncated, info = env.step(action)

        done = terminated or truncated

        q_table[state, action] = q_table[state, action] + alpha * (reward + gamma * np.max(q_table[next_state]) - q_table[state, action])

        state = int(next_state)

    if episode % 100 == 0:
        raw = env.render()
        print(render(raw))

    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    print(f"Episode : {episode + 1}, Steps : {steps}")
    print("-------------------------")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
-------------------------
Episode : 7605, Steps : 11
-------------------------
Episode : 7606, Steps : 17
-------------------------
Episode : 7607, Steps : 13
-------------------------
Episode : 7608, Steps : 8
-------------------------
Episode : 7609, Steps : 13
-------------------------
Episode : 7610, Steps : 13
-------------------------
Episode : 7611, Steps : 18
-------------------------
Episode : 7612, Steps : 12
-------------------------
Episode : 7613, Steps : 16
-------------------------
Episode : 7614, Steps : 14
-------------------------
Episode : 7615, Steps : 12
-------------------------
Episode : 7616, Steps : 15
-------------------------
Episode : 7617, Steps : 25
-------------------------
Episode : 7618, Steps : 14
-------------------------
Episode : 7619, Steps : 17
-------------------------
Episode : 7620, Steps : 7
-------------------------
Episode : 7621, Steps : 21
-------------------------
Episode : 7622, Steps : 19
------------

In [137]:
# test

rewards_list = []

for episode in range(1000):
    state, info = env.reset()
    state = int(state)
    step = 0
    done = False
    rewards = 0

    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, terminated, truncated, info = env.step(action)
        rewards += int(reward)

        done = terminated or truncated
        state = int(next_state)
        step += 1

    rewards_list.append(rewards)

    raw = env.render()
    print(render(raw))

    print(f"Episode : {episode}, Steps : {step}")

env.close()

print(f"Average reward : {np.mean(rewards_list)}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 500, Steps : 16
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 501, Steps : 13
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 502, Steps : 14
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 503, Steps : 11
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 504, Steps : 11
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 505, Steps : 12
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 506, Steps : 13
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode : 507, Ste

In [138]:
q_table

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-1.73338468, -0.11888383, -1.66811208, -0.34547234,  1.62261467,
        -8.42141442],
       [ 3.4930129 ,  4.48397214,  2.05623723,  3.97618209,  7.7147    ,
        -3.95603944],
       ...,
       [-1.20461593,  5.30628296, -1.20468908,  0.13082405, -1.931761  ,
        -4.16005318],
       [-2.33152962, -2.60297659, -2.58176241,  1.49763649, -5.52106144,
        -2.82858636],
       [-0.446941  ,  1.62825091, -0.36748   , 16.42175469, -3.45529   ,
        -1.        ]])

학습률 낮게 설정하고 에피소드 적을 때 보상 낮음 : -200

에피소드 1000 -> 10000 : -112까지 오름

학습률 0.01 -> 0.1 : 양수로 오름
